In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install lorem
!pip install geonamescache
!pip install random2

In [ ]:
import pandas as pd
import numpy as np
import geonamescache
import random
import lorem
import csv

# Load data

In [ ]:
article_header = pd.read_csv("/content/drive/MyDrive/UPC/SDM/Project1/data/dblp_article_header.csv", sep=";")
article = pd.read_csv("/content/drive/MyDrive/UPC/SDM/Project1/data/dblp_article.csv", sep=";", header=0, names=article_header.columns)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,4,5,6,7,8,9,10,11,13,14,18,19,20,21,22,23,24,25,26,30,31,33) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
cite_has_citations = pd.read_csv("/content/drive/MyDrive/UPC/SDM/Project1/data/dblp_cite_has_citation.csv", sep=";")
# :START_ID - paper
# :END_ID - paper
all_ref_papers = list(set(list(cite_has_citations[':START_ID']) + list(cite_has_citations[':END_ID'])))
# cite_has_citations.to_csv('/content/drive/MyDrive/UPC/SDM/Project1/output/cite_has_citation.csv',index=False)
# all_ref_papers2 = np.unique(citations[':START_ID'].values + citations[':END_ID'].values) # my way doesn't work TODO

**Properties for the article node is prepared** 

In [ ]:
# Taking the articles which have citations or either being cited
articles_list = article[article['article:ID'].isin(all_ref_papers)]
#keep columns those are useful
column_list=['article:ID','key:string', 'mdate:date','title:string','volume:string', 'year:int']
articles_list=articles_list[column_list]
articles_list=articles_list.dropna()

articles_list['year:int']=articles_list['year:int'].astype(int)
articles_list.head()


,article:ID,key:string,mdate:date,title:string,volume:string,year:int
111029,6252633,journals/ras/PortugalCGSP16,2020-02-24,"Retraction notice to: ""An autonomous all terra...",77,2016
156870,6298474,journals/eswa/LiH15,2017-05-26,"Retraction notice to ""Applying TRIZ and Fuzzy ...",42,2015
166554,6308158,journals/eswa/Kaya12,2017-05-26,"Retraction notice to ""A genetic algorithm appr...",39,2012
167349,6308953,journals/eswa/Viejo016,2017-05-26,"Removal notice to ""Enforcing Transparent Acces...",63,2016
170171,6311775,journals/eswa/NooriAFA16,2017-05-26,"Retraction notice to ""Results uncertainty of s...",63,2016


In [ ]:

author = pd.read_csv("/content/drive/MyDrive/UPC/SDM/Project1/data/dblp_author.csv", sep=";")
author=author.rename(columns={":ID": "author_id:ID"})
#authors_articles = pd.read_csv('Data/dblp_author_authored_by.csv', sep=';')
author_authored_by = pd.read_csv("/content/drive/MyDrive/UPC/SDM/Project1/data/dblp_author_authored_by.csv", sep=";")
# :START_ID - article id
# :END_ID - author ( authored by )
# article - [authored] -> author
#author_autohred_by.head()
author_authored_by=author_authored_by.rename(columns={":START_ID": "article:ID",":END_ID": "author_id:ID"})
author_authored_by.head()


,article:ID,author_id:ID
0,3,8970872
1,5878537,8970872
2,7736682,8970872
3,416341,8970872
4,5981815,8970872


Article

In [ ]:
# From the cited articles only get the authors that authored the articles
authors_articles_list = author_authored_by[author_authored_by['article:ID'].isin(list(articles_list['article:ID']))==True]
authors_articles_list.head()
# change direction. Instead of (article)-[authored by]->(author), (author)-[writes]->(article)
#authors_articles_list.rename(columns={'author_id':':START_ID','article_id':':END_ID'}, inplace=True)


,article:ID,author_id:ID
648,8918914,8970892
678,8919619,8970892
704,8069620,8970892
719,8917428,8970893
723,8069438,8970893


**Authors node created**

In [ ]:
#article[article['article:ID'].isin(all_ref_papers) == False]
# Getting the authors and author_id that are cited
author_list = author[author['author_id:ID'].isin(list(authors_articles_list['author_id:ID']))== True]
author_list.to_csv('/content/drive/MyDrive/UPC/SDM/Project1/output/authors.csv',index=False)
author_list

,author_id:ID,author:string
20,8970892,Asit Dan
21,8970893,David B. Kemp
27,8970899,Leo Mark
30,8970902,Heping Shang
35,8970907,Inderpal Singh Mumick
...,...,...
3004767,11975639,Sebastiaan H. von Solms
3004783,11975655,David W. Stemple
3004796,11975668,Ambuj K. Singh
3004805,11975677,Uri Shaft


In [ ]:
#Creating journal and conference csv
journals = pd.read_csv('/content/drive/MyDrive/UPC/SDM/Project1/data/dblp_journal.csv',sep=';')
journals=journals.rename(columns={":ID": "journal_id:ID"})

# getting the journals that have articles
articles_in_journals = pd.read_csv('/content/drive/MyDrive/UPC/SDM/Project1/data/dblp_journal_published_in.csv',sep=';')
articles_in_journals = articles_in_journals.rename(columns={":START_ID": "article:ID",":END_ID": "journal_id:ID"})
articles_in_journals = articles_in_journals[ articles_in_journals['journal_id:ID'].isin(list(journals['journal_id:ID']))==True]
#start_id=article_id and end_id=journal_id

articles_in_journals_list = pd.merge(articles_list[['article:ID','volume:string','year:int','mdate:date']],articles_in_journals,on='article:ID')
#articles_in_journals_list.to_csv('/content/drive/MyDrive/UPC/SDM/Project1/output/article_publishedin_journal.csv',index=False)
articles_in_journals_list

,article:ID,volume:string,year:int,mdate:date,journal_id:ID
0,6252633,77,2016,2020-02-24,12075629
1,6298474,42,2015,2017-05-26,12075675
2,6308158,39,2012,2017-05-26,12075675
3,6308953,63,2016,2017-05-26,12075675
4,6311775,63,2016,2017-05-26,12075675
...,...,...,...,...,...
1835,8920458,7,1995,2017-11-06,12077408
1836,8920467,4,1992,2017-05-20,12077408
1837,8920470,3,1991,2017-05-20,12077408
1838,8920472,3,1991,2020-06-15,12077408


**Individual journal and conference node is created along with the associated relations with articles**

In [ ]:
# conference information - because we don't have conferences, we can use the journals and modify it.
# we get the  
# articles_in_conferences.rename(columns={"journal_id": "conference_id"}, inplace=True)
#articles_in_journals_list=articles_in_journals_list.rename(columns={"article_id": ":START_ID","journal_id": ":END_ID"})

half = len(articles_in_journals_list) // 2
articles_in_conferences_list = articles_in_journals_list.iloc[:half, :]
articles_in_journals_list = articles_in_journals_list.iloc[half:, :]
journal_list=journals[journals['journal_id:ID'].isin(list(articles_in_journals_list['journal_id:ID']))==True]
conf_list=journals[journals['journal_id:ID'].isin(list(articles_in_conferences_list['journal_id:ID']))==True]
journal_list.to_csv('/content/drive/MyDrive/UPC/SDM/Project1/output/journal.csv',index=False)
conf_list=conf_list.rename(columns={"journal:string":"conference:string"})
conf_list.to_csv('/content/drive/MyDrive/UPC/SDM/Project1/output/conference.csv',index=False)

articles_in_journals_list=articles_in_journals_list.rename(columns={"article:ID": ":START_ID","journal_id:ID": ":END_ID"})
articles_in_journals_list.to_csv('/content/drive/MyDrive/UPC/SDM/Project1/output/article_publishedin_journal.csv',index=False)
articles_in_conferences_list.rename(columns={"article:ID": ":START_ID" ,"volume:string": "edition:string","journal_id:ID": ":END_ID",}, inplace=True)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
conf_list

,journal_id:ID,conference:string
1,12075516,SIGMOD Rec.
114,12075629,Robotics Auton. Syst.
160,12075675,Expert Syst. Appl.
300,12075815,Pattern Recognit. Lett.
349,12075864,J. Biomed. Informatics
439,12075954,Signal Process.
468,12075983,J. Symb. Log.
472,12075987,Comput. Math. Appl.
481,12075996,IEEE Data Eng. Bull.
534,12076049,Comput. Educ.


In [ ]:
# conference/workshop is organized in terms of editions. Each edition of a conference is held
# in a given city (venue) at a specific period of time of a given year.

gc = geonamescache.GeonamesCache()
countries = list(gc.get_us_states_by_names().keys())
# print countries dictionary
conference_countries = [random.choice(countries) for i in range(half) ]
articles_in_conferences_list['venues'] = conference_countries
articles_in_conferences_list.head()
articles_in_conferences_list.to_csv('/content/drive/MyDrive/UPC/SDM/Project1/output/article_publishedin_conference.csv',index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#making the first author as corresponding author
authors_articles_list.sort_values(by='article:ID', inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


**Author article relation is created**

In [ ]:
# Making the first author as corresponding author
authors_articles_list['corres_author']=(~authors_articles_list.duplicated(['article:ID']))
authors_articles_list=authors_articles_list.iloc[:, [1,0,2]]
author
authors_articles_set=authors_articles_list.rename(columns={"article:ID":":START_ID", "author_id:ID":":END_ID"})
authors_articles_set.to_csv('/content/drive/MyDrive/UPC/SDM/Project1/output/author_writes_article.csv',index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
authors_articles_list

,author_id:ID,article:ID,corres_author
1503873,9054297,6759977,True
18919414,11552922,6760004,True
6417711,9700670,6760004,False
1512558,9054356,6760004,False
1134166,9041649,6760004,False
...,...,...,...
4057,8970975,8920458,False
8567004,10036941,8920467,True
4553144,9390013,8920467,False
16489827,11150040,8920470,True


**Article node is created with review attribute synthetically generated**

In [ ]:
# Adding abstract to the selected articles
abs_list = []
for i in range(articles_list.shape[0]):
    abs_list.append(lorem.paragraph())
articles_list['abstract'] = abs_list
articles_list.to_csv('/content/drive/MyDrive/UPC/SDM/Project1/output/articles.csv',index=False)

**Keywords node is  created**

In [ ]:
#Adding keywords
key_list=["Database","Data Management","Indexing","Data Structure","Algorithm","Machine Learning","AI","NLP"]
key_df=pd.DataFrame()
key_id_list=[i for i in range(len(key_list))]
key_df['key:ID']=key_id_list
key_df['keywords']=key_list
key_df.to_csv('/content/drive/MyDrive/UPC/SDM/Project1/output/keywords.csv',index=False)


**Relation with keywords and articles created**

In [ ]:
# We are assuming an article can have at most 5 keywords
with open('/content/drive/MyDrive/UPC/SDM/Project1/output/article_keywords.csv', 'w', newline='') as writeFile:
    writer = csv.writer(writeFile)
    #start_id=article_id , end_id=key_id
    writer.writerow([':START_ID',':END_ID'])
    art_list=articles_list["article:ID"].tolist()
    for articles in art_list:
      num_key=random.randint(1,5)
      for i in range(num_key):
        key_name=random.randint(0,len(key_list)-1)
        writer.writerow([articles,key_name])

In [ ]:
from collections import defaultdict

# first we have to find the papers of each author.
art_auth = defaultdict(list)
for idx,row in authors_articles_list.iterrows():
    art_auth[row['article:ID']].append(row['author_id:ID'])

In [ ]:
article_set=set(authors_articles_list['article:ID'].tolist())
author_set=set(authors_articles_list['author_id:ID'].tolist())

**article reviewedby author relation is created**

In [ ]:
with open('/content/drive/MyDrive/UPC/SDM/Project1/output/article_reviewedby_author.csv', 'w', newline='') as writeFile:
    writer = csv.writer(writeFile)
    #start_id=author_id, end_id=article_id
    writer.writerow([':START_ID','review',':END_ID'])
    for articles in article_set:
      tmp_author=author_set-set(art_auth)
      for i in range (3):
        review=lorem.sentence()
        tmp_auth_id=random.choice(list(tmp_author))
        writer.writerow([articles,review,tmp_auth_id])

    

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/UPC/SDM/Project1/output/article_reviewedby_author.csv')
df

,:START_ID,review,:END_ID
0,8069122,Modi velit sit magnam dolor quiquia velit.,10179851
1,8069122,Ut tempora aliquam velit modi.,11975315
2,8069122,Sit neque ut quisquam adipisci aliquam numquam...,9935862
3,8069123,Est sit quaerat consectetur non.,11852475
4,8069123,Quisquam magnam tempora porro dolore sit dolor...,10671416
...,...,...,...
5437,7790565,Tempora amet quisquam quiquia voluptatem conse...,9886474
5438,7790565,Magnam neque ut adipisci quisquam consectetur.,11113477
5439,8069110,Dolorem est quisquam labore.,9059369
5440,8069110,Ipsum tempora ut dolor.,11974950


**the article citation relation is created**

In [ ]:
# Create the citation relationship
# article:ID - cites - article:ID
# An article can cite another article

# we use the articles that already have the citations and randomly 

article_cites = {}

for random_article in articles_list['article:ID'].sample(200).values:
  nb_of_citations = random.randint(1,10)
  citation_list = articles_list['article:ID'].sample(nb_of_citations).values
  if(random_article not in citation_list):
    article_cites[random_article] = citation_list

a = []
b = []

for k, v in article_cites.items():
  for el in v:
    a.append(k)
    b.append(el)

cited_articles = pd.DataFrame({':START_ID': a, ':END_ID': b})
cited_articles.head()

,:START_ID,:END_ID
0,8918026,7846615
1,8918026,8919305
2,8918026,8920043
3,8918026,8611019
4,8918026,7846603


In [ ]:
cited_articles.to_csv('/content/drive/MyDrive/UPC/SDM/Project1/output/article_cites.csv')

##**Evolving the graph: Adding affiliations**

**University and company nodes are cretaed**

In [ ]:

uni_list=["UCLA","ULB","VUB","DIU","MIT","HARVARD","UALG","UAB","URIT","RMIT"]
comp_list=["Microsoft","Google","Vanderlande","STIB","BEXIMCO","TATA","LG","PORTER","BKASH","PATHAO"]
uni_df=pd.DataFrame()
uni_id_list=[i for i in range(len(uni_list))]
uni_df['uni:ID']=uni_id_list
uni_df['name:string[]']=uni_list
uni_df.to_csv('/content/drive/MyDrive/UPC/SDM/Project1/output/university.csv',index=False)

com_df=pd.DataFrame()
com_id_list=[i for i in range(len(comp_list))]
com_df['com:ID']=com_id_list
com_df['name:string[]']=comp_list
com_df.to_csv('/content/drive/MyDrive/UPC/SDM/Project1/output/company.csv',index=False)



**Affiliation with the university relation created **

In [ ]:
with open('/content/drive/MyDrive/UPC/SDM/Project1/output/affiliation_university.csv', 'w', newline='') as writeFile:
    writer = csv.writer(writeFile)
    #start_id=author_id , end_id=uni_id
    writer.writerow([':START_ID',':END_ID'])
    auth_list=author_list["author_id:ID"].tolist()
    for auth in auth_list:
      key_name=random.randint(0,15)
      if key_name<10:
        writer.writerow([auth,key_name])
      else:
        c=1

**Affiliation with the company relation created**

In [ ]:
with open('/content/drive/MyDrive/UPC/SDM/Project1/output/affiliation_company.csv', 'w', newline='') as writeFile:
    writer = csv.writer(writeFile)
    #start_id=author_id , end_id=com_id
    writer.writerow([':START_ID',':END_ID'])
    auth_list=author_list["author_id:ID"].tolist()
    for auth in auth_list:
      key_name=random.randint(0,15)
      if key_name<10:
        writer.writerow([auth,key_name])
      else:
        c=1

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/UPC/SDM/Project1/output/company.csv')
df

,com:ID,name:string[]
0,0,Microsoft
1,1,Google
2,2,Vanderlande
3,3,STIB
4,4,BEXIMCO
5,5,TATA
6,6,LG
7,7,PORTER
8,8,BKASH
9,9,PATHAO
